In [50]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import librosa
import soundfile as sf
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm import tqdm
import musdb
from neural_net import UNet
from torch.optim.lr_scheduler import OneCycleLR
from torch.cuda.amp import autocast, GradScaler
# ── Hyperparameters & Paths ───────────────────────────────────────────────────
N_FFT         = 1024
HOP_LENGTH    = 256
MAX_FRAMES    = 512
OVERLAP       = 64
BATCH_SIZE    = 16
LR            = 1e-3
EPOCHS        = 50
DATA_DIR      = "musdb18wavs/train"
MODEL_PATH    = "models/vocal_separator_unet.pt"

DEVICE = (
    torch.device("cuda")
    if torch.cuda.is_available()
    else torch.device("mps")
    if torch.backends.mps.is_available()
    else torch.device("cpu")
)
print(f"➡️ Using device: {DEVICE}")

➡️ Using device: mps


In [ ]:
def convert_tracks_to_wav():
    """
        convert the musdb18 dataset to .wav format
    """
    for track in mus.tracks:
        track_name = track.name.replace(" ", "")
        train_test = track.subset
        track_folder = os.path.join(f"musdb18wavs/{train_test}", track_name)
        os.makedirs(track_folder, exist_ok=True)
        mixture_path = os.path.join(track_folder, "mixture.wav")
        sf.write(mixture_path, track.audio, track.rate)
        for source in ["vocals", "drums", "bass", "other"]:
            if source in track.targets:
                source_audio = track.targets[source].audio
                source_path = os.path.join(track_folder, f"{source}.wav")
                sf.write(source_path, source_audio, track.rate)    


In [4]:
def get_stft(path):
    y, sr = librosa.load(path, sr=None)
    stft = librosa.stft(y, n_fft=N_FFT, hop_length=HOP_LENGTH, window='hann')
    return stft, sr

In [5]:
def istft(stft):
    return librosa.istft(stft, hop_length=HOP_LENGTH)

In [ ]:
def display_stft(stft_matrix, title):
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(stft_matrix), ref=np.max),
                             y_axis='log', x_axis='time')
    plt.title(title)
    plt.colorbar(format='%+2.0f dB')
    plt.tight_layout()
    plt.show()

In [51]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
import librosa

class RemoveVocalsDataset(Dataset):
    def __init__(self, base_dir):
        self.segments = []
        step = MAX_FRAMES - OVERLAP

        for track in os.listdir(base_dir):
            mix_p   = os.path.join(base_dir, track, "mixture.wav")
            vocal_p = os.path.join(base_dir, track, "vocals.wav")
            if not (os.path.exists(mix_p) and os.path.exists(vocal_p)):
                continue

            # load mono audio
            mix_y,  _ = librosa.load(mix_p,   sr=None, mono=True)
            voc_y,  _ = librosa.load(vocal_p, sr=None, mono=True)

            # STFT → magnitude
            mix_stft = librosa.stft(mix_y,  n_fft=N_FFT, hop_length=HOP_LENGTH, window="hann")
            voc_stft = librosa.stft(voc_y,  n_fft=N_FFT, hop_length=HOP_LENGTH, window="hann")
            mix_mag  = np.abs(mix_stft)
            voc_mag  = np.abs(voc_stft)

            # background magnitude & mask
            bg_mag = np.clip(mix_mag - voc_mag, 0.0, None)
            mask   = np.clip(bg_mag / (mix_mag + 1e-8), 0.0, 1.0)

            # log‐scale mixture input
            mix_log = np.log1p(mix_mag)

            T = mix_log.shape[1]
            for t0 in range(0, T - MAX_FRAMES + 1, step):
                m_chunk    = mix_log[:,   t0 : t0 + MAX_FRAMES]
                mask_chunk = mask    [:,   t0 : t0 + MAX_FRAMES]

                self.segments.append((
                    torch.from_numpy(m_chunk).unsqueeze(0).float(),    # [1, freq, frames]
                    torch.from_numpy(mask_chunk).unsqueeze(0).float()  # [1, freq, frames]
                ))

    def __len__(self):
        return len(self.segments)

    def __getitem__(self, idx):
        return self.segments[idx]


In [31]:
class SeparationDataset(Dataset):
    def __init__(self, base_dir):
        self.segments = []
        step = MAX_FRAMES - OVERLAP
        for track in os.listdir(base_dir):
            mix_p   = os.path.join(base_dir, track, "mixture.wav")
            vocal_p = os.path.join(base_dir, track, "vocals.wav")
            if not (os.path.exists(mix_p) and os.path.exists(vocal_p)):
                continue
            mix_y,   _ = librosa.load(mix_p, sr=None, mono=True)
            vocal_y, _ = librosa.load(vocal_p, sr=None, mono=True)
            mix_stft   = librosa.stft(mix_y,   n_fft=N_FFT, hop_length=HOP_LENGTH, window='hann')
            vocal_stft = librosa.stft(vocal_y, n_fft=N_FFT, hop_length=HOP_LENGTH, window='hann')
            mix_mag    = np.abs(mix_stft)
            vocal_mag  = np.abs(vocal_stft)

            T = mix_mag.shape[1]
            for t0 in range(0, T - MAX_FRAMES + 1, step):
                m_chunk = mix_mag[:,  t0:t0+MAX_FRAMES]
                v_chunk = vocal_mag[:,t0:t0+MAX_FRAMES]
                is_silent = (v_chunk.sum() < 1e-3)
                if is_silent and np.random.rand() > 0.5:
                    continue

                mask    = np.clip(v_chunk / (m_chunk + 1e-8), 0.0, 1.0)
                mix_log = np.log1p(m_chunk)
                self.segments.append((
                    torch.from_numpy(mix_log).unsqueeze(0).float(),
                    torch.from_numpy(mask).unsqueeze(0).float()
                ))

    def __len__(self):
        return len(self.segments)
    def __getitem__(self, idx):
        return self.segments[idx]

In [ ]:
def spectral_convergence(true_spec, est_spec):
    true_mag = true_spec.abs()
    est_mag  = est_spec.abs()
    num      = torch.norm(true_mag - est_mag, p='fro')
    den      = torch.norm(true_mag,           p='fro')
    return num / (den + 1e-8)

def magnitude_loss(true_spec, est_spec):
    return F.l1_loss(est_spec.abs(), true_spec.abs())

def waveform_loss(true_wave, est_wave):
    return F.l1_loss(est_wave, true_wave)

In [52]:
full_ds      = RemoveVocalsDataset(DATA_DIR)
indices      = torch.randperm(len(full_ds))[:1000].tolist()
subset_ds    = Subset(full_ds, indices)
loader       = DataLoader(full_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)

In [53]:
model2 = UNet().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = nn.L1Loss()

for epoch in range(1, EPOCHS + 1):
    model.train()
    running_loss = 0.0

    for mix_log, mask_tgt in tqdm(loader, desc=f"Epoch {epoch}/{EPOCHS}"):
        mix_log  = mix_log.to(DEVICE)
        mask_tgt = mask_tgt.to(DEVICE)

        pred = model(mix_log)
        loss = criterion(pred, mask_tgt)

        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(loader)
    print(f"Epoch {epoch} avg loss: {avg_loss:.4f}")

Epoch 1/50: 100%|██████████| 546/546 [03:19<00:00,  2.73it/s]


Epoch 1 avg loss: 0.2466


Epoch 2/50: 100%|██████████| 546/546 [03:12<00:00,  2.83it/s]


Epoch 2 avg loss: 0.2373


Epoch 3/50: 100%|██████████| 546/546 [03:13<00:00,  2.83it/s]


...

Epoch 48 avg loss: 0.1887


Epoch 49/50: 100%|██████████| 546/546 [03:24<00:00,  2.67it/s]


Epoch 49 avg loss: 0.1898


Epoch 50/50: 100%|██████████| 546/546 [03:21<00:00,  2.71it/s]


Epoch 50 avg loss: 0.1887


In [35]:
model = UNet().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = nn.L1Loss()

for epoch in range(1, EPOCHS + 1):
    model.train()
    running_loss = 0.0

    for mix_log, mask_tgt in tqdm(loader, desc=f"Epoch {epoch}/{EPOCHS}"):
        mix_log  = mix_log.to(DEVICE)
        mask_tgt = mask_tgt.to(DEVICE)

        pred = model(mix_log)
        loss = criterion(pred, mask_tgt)

        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(loader)
    print(f"Epoch {epoch} avg loss: {avg_loss:.4f}")

Epoch 1/100: 100%|██████████| 505/505 [02:55<00:00,  2.88it/s]


Epoch 1 avg loss: 0.3418


Epoch 2/100: 100%|██████████| 505/505 [02:51<00:00,  2.94it/s]


Epoch 2 avg loss: 0.2957


Epoch 3/100: 100%|██████████| 505/505 [02:55<00:00,  2.87it/s]


...

Epoch 98 avg loss: 0.2146


Epoch 99/100: 100%|██████████| 505/505 [03:04<00:00,  2.73it/s]


Epoch 99 avg loss: 0.2150


Epoch 100/100: 100%|██████████| 505/505 [03:06<00:00,  2.70it/s]

Epoch 100 avg loss: 0.2148


In [ ]:
    model = UNet().to(DEVICE)
    state = torch.load("/Users/drewmedina/Documents/GitHub/AI-Stem-Separation/models/vocal_separator_unet6.pth", map_location=DEVICE)
    model.load_state_dict(state)
    model.to(DEVICE)
    model.eval()

/var/folders/kd/_03l76p11_xc3fy9bn3k0_wr0000gn/T/ipykernel_38602/3796454912.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load("/Users/drewmedina/Documen

UNet(
  (encoder): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (middle): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [56]:
import numpy as np
import torch
import librosa
import soundfile as sf

def separate_vocals(mixture_path, output_path="predicted_vocals.wav"):
    S, sr = get_stft(mixture_path)
    mag   = np.abs(S)
    phase = np.angle(S)
    T_tot    = mag.shape[1]
    pred_mag = np.zeros_like(mag)
    counts   = np.zeros(T_tot)
    step     = MAX_FRAMES - OVERLAP
    t        = 0
    while t + MAX_FRAMES <= T_tot:
        seg     = mag[:, t : t + MAX_FRAMES]            # (513,256)
        mix_log = np.log1p(seg)[None, None, :, :]       # (1,1,513,256)
        m_t = torch.from_numpy(mix_log).float().to(DEVICE)
        with torch.no_grad():
            out  = model(m_t)                           # (1,1,513,256)
            pred = out.squeeze(0).squeeze(0).cpu().numpy()  # (513,256)
        pred_mag[:, t : t + MAX_FRAMES] += seg * pred
        counts[  t : t + MAX_FRAMES] += 1
        t += step
    if t < T_tot:
        rem = mag[:, t:]
        pad = MAX_FRAMES - rem.shape[1]
        seg = np.pad(rem, ((0,0),(0,pad)), mode="constant")
        mix_log = np.log1p(seg)[None, None, :, :]
        m_t     = torch.from_numpy(mix_log).float().to(DEVICE)
        with torch.no_grad():
            out  = model(m_t)                          
            pred = out.squeeze(0).squeeze(0).cpu().numpy()[:, : rem.shape[1]]
        pred_mag[:, t:] += rem * pred
        counts[t:]     += 1
    counts = np.maximum(counts, 1.0)
    pred_mag /= counts[None, :]
    S_est = pred_mag * np.exp(1j * phase)
    y_est = istft(S_est)
    sf.write(output_path, y_est, sr)
    print(f"🎉 Saved separated vocals to {output_path}")
separate_vocals("figures (mastered).wav")


🎉 Saved separated vocals to predicted_vocals.wav


In [ ]:
MODEL_SAVE_PATH = "models/vocal_separator_unet5.pth"
os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True)
torch.save(model.state_dict(), MODEL_SAVE_PATH)
print(f"Model saved to {MODEL_SAVE_PATH}")


In [26]:
import museval
mus = musdb.DB(root="musdb18", subsets="test")
# we'll collect all track‐wise scores here
results = museval.EvalStore(
    frames_agg='median',
    tracks_agg='median'
)
for i, track in enumerate(mus.tracks):
    print(f"▶ Processing {track}")
    track_path = f"musdb18wavs/test/{track}/mixture.wav"
    track_path = track_path.replace(" ", "")
    # 1) write out your model's vocal estimate
    out_vocals = os.path.join("tests", f"{track}__vocals.wav")
    separate_vocals(track_path, out_vocals)

    # 2) load back the estimate & the original mixture
    mix, _       = sf.read(track_path,   always_2d=True)  # → (nsamples, nch)
    est_vocals, _ = sf.read(out_vocals, always_2d=True)   # → (nsamples, nvoc_ch)
    # if your model output is mono, nvoc_ch==1, so tile it to stereo:
    if est_vocals.shape[1] == 1 and mix.shape[1] == 2:
        est_vocals = np.repeat(est_vocals, 2, axis=1)     # → (nsamples, 2)

    # now subtraction works
    est_accom = mix - est_vocals  
    estimates = {
        'vocals'       : est_vocals,
        'accompaniment': est_accom

    }

    # 4) compute BSSeval-v4 metrics for this track
    scores = museval.eval_mus_track(
        track,
        estimates,
        output_dir=None     # don't need the intermediate JSON files
    )
    results.add_track(scores)
    if i == 5:
        break

# ——— Aggregate & pretty-print ————————————————————————————————
print("\n\n===== AGGREGATED RESULTS =====\n")
print(results)  # prints something like your matrix: µ ± σ for SDR, SIR, SAR, ISR, etc.

# if you want the raw pandas DataFrame for further processing:
df = results.save('my_method.pandas')  # returns a DataFrame of all metrics
print("\n\nRaw DataFrame:\n", df)

▶ Processing AM Contra - Heart Peripheral
🎉 Saved separated vocals to tests/AM Contra - Heart Peripheral__vocals.wav
▶ Processing Al James - Schoolboy Facination
🎉 Saved separated vocals to tests/Al James - Schoolboy Facination__vocals.wav
▶ Processing Angels In Amplifiers - I'm Alright
🎉 Saved separated vocals to tests/Angels In Amplifiers - I'm Alright__vocals.wav
▶ Processing Arise - Run Run Run
🎉 Saved separated vocals to tests/Arise - Run Run Run__vocals.wav
▶ Processing BKS - Bulldozer
🎉 Saved separated vocals to tests/BKS - Bulldozer__vocals.wav
▶ Processing BKS - Too Much
🎉 Saved separated vocals to tests/BKS - Too Much__vocals.wav


===== AGGREGATED RESULTS =====

Aggrated Scores (median over frames, median over tracks)
vocals          ==> SDR:   2.923  SIR:   3.322  ISR:   7.139  SAR:   6.840  
accompaniment   ==> SDR:   7.489  SIR:  11.241  ISR:  10.775  SAR:  11.854  



Raw DataFrame:
 None
